# Data Generation

Step 1. Find the HR images on your file system and add them a list of string/Path

In [ ]:
from pathlib import Path

from datagen import dataset_generation

trainings_nii_paths = []


##### TODO ADD you training niis ######
dataset_name = "my_dataset"
name_folder = "sag"  # "ax"
sr_target = 0.8571  # 0.5
root = "/media/data/robert/datasets/dataset-neuropoly/"
for i in Path(root).rglob(f"*/*_acq-{name_folder}_*T2w.nii.gz"):
    if "manual" in str(i):
        continue
    if "stitched" in str(i):
        continue
    trainings_nii_paths.append(i)
########################################

Step 2 make a jpg dataset

In [ ]:
if name_folder == "ax":
    dataset_generation.make_ds(
        dataset=trainings_nii_paths,
        acq="ax",
        name_folder=name_folder,
        orientation=("I", "R", "P"),  # First letter is where we cut in to slices, Second it to upscale plane
        zoom=(-1, sr_target, sr_target),
        out=f"dataset/{dataset_name}/",
    )
else:
    ## Sag
    dataset_generation.make_ds(
        dataset="trainings_nii_paths",
        acq="sag",
        name_folder=name_folder,
        orientation=("R", "I", "P"),
        zoom=(-1, sr_target, sr_target),
        out=f"/dataset/{dataset_name}/",
    )

In [ ]:
sr_source = None
if name_folder == "ax":
    sr_source = [2.5 + i / 10 for i in range(30)]
config = f"""# Training
experiment_name: {dataset_name}
lr: 0.0001
batch_size:  32
target_batch_factor: 8
batch_size_eval:32
total_samples:50_000_000
batch_size_eval:  10
gpus: [0]
num_cpu: 16
# Logging
log_dir: "lightning_logs_dae"
fast_dev_run : False
optimizer : adam
weight_decay:  0.0
dataset :"dataset/{dataset_name}/train/{name_folder}"
dataset_val: "dataset/{dataset_name}/val/{name_folder}"
in_channels:  1  # Channel of the Noised input
img_size: 160
dims:  2
monitor: "train/avg_loss"
# Dataset
ds_type: "csv_2D_super"
x_start: "hq"
image_name : "img_lr"
palette_condition: ["img_lr"]
sr_source: {sr_source}
sr_target:{sr_target}
## Other by Model ##
train_mode : "diffusion"
model_type : "palette_only"
ablation_lvl: 11
"""
config_f = f"config/{dataset_name}.conf"

with open(config_f, "w") as file:
    file.write(config)
print(f"you can now start the training with:\n python train_DAE.py --config {config_f}")